#Labeling Stats Dashboard
### below cells contains the DB SQL queries which is used to create the Statistical Visualizations in DBX Dashboard


### Information about the catalog, schema, and tables which are used in the Queries

- **Catalog**: `enhanced_feedback`  
- **Schema 1**: `bronze` → Table: `human_label_parsed`  
- **Schema 2**: `silver` → Table: `customer_feedback`  
- **Schema 3**: `gold` → Table: `email_digest`


# Dashboard : Labeling Stats
### Query 1: Team Contributions for current week

In [0]:
WITH team_counts AS (
    
    SELECT
        'Labeling team contribution' AS team, 
        COUNT(*) AS count_value 
    FROM enhanced_feedback.bronze.human_label_parsed
    WHERE human_labels.Username IN ('adeshnur','deb', 'rkanase', 'mbalaraddi', 'smeti', 'mkoujalagi', 'srshetty', 'pred', 'ras', 'msheeri', 'avpatil')
        AND approximateArrivalTimestamp IS NOT NULL
        
        AND DATE(approximateArrivalTimestamp) BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1)  -- Sunday of the current week
                                             AND LEAST(CURRENT_DATE, DATE_ADD(DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1), 6))  -- up to today or Saturday
    UNION ALL

    
    SELECT
        'BRTS team contribution' AS team, 
        COUNT(*) AS count_value 
    FROM enhanced_feedback.bronze.human_label_parsed
    WHERE human_labels.Username IN ('mshettar', 'skotyalkar', 'usharma', 'spoomalai', 'psethy', 'gkenja', 'stalekar', 'pob', 'aisk', 'vprasad', 'jkumar')
        AND approximateArrivalTimestamp IS NOT NULL
        
        AND DATE(approximateArrivalTimestamp) BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1)  -- Sunday of the current week
                                             AND LEAST(CURRENT_DATE, DATE_ADD(DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1), 6))  -- up to today or Saturday
),

LabelTotals AS (
    SELECT
        team,
        count_value
    FROM team_counts

    UNION ALL

    SELECT
        'Total' AS team, 
        SUM(count_value) AS count_value 
    FROM team_counts
)

SELECT
    team, 
    count_value 
FROM LabelTotals 

UNION ALL


SELECT
    'Fully classified emails' AS team, 
    COUNT(DISTINCT essMessageId) AS count_value 
from enhanced_feedback.silver.customer_feedback 
WHERE
    labelData.human_label_count >= 3
   and date(modificationTime) IS NOT NULL
    AND DATE(modificationTime) BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1)  -- Sunday of the current week
    AND LEAST(CURRENT_DATE, DATE_ADD(DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1), 6)); -- up to today or Saturday

### Query 2 : Team Contributions for previous week

In [0]:
WITH team_counts AS (
    
    SELECT
        'Labeling team contribution' AS team, 
        COUNT(*) AS count_value 
    FROM enhanced_feedback.bronze.human_label_parsed
    WHERE human_labels.Username IN ('adeshnur','deb', 'rkanase', 'mbalaraddi', 'smeti', 'mkoujalagi', 'srshetty', 'pred', 'ras', 'msheeri', 'avpatil')
        AND approximateArrivalTimestamp IS NOT NULL
        AND DATE(approximateArrivalTimestamp) BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 8) -- Previous Sunday
                                             AND DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 2) -- Previous Saturday

    UNION ALL

    
    SELECT
        'BRTS team contribution' AS team, 
        COUNT(*) AS count_value 
    FROM enhanced_feedback.bronze.human_label_parsed
    WHERE human_labels.Username IN ('mshettar', 'skotyalkar', 'usharma', 'spoomalai', 'psethy', 'gkenja', 'stalekar', 'pob', 'aisk', 'vprasad', 'jkumar')
        AND approximateArrivalTimestamp IS NOT NULL
        AND DATE(approximateArrivalTimestamp) BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 8) -- Previous Sunday
                                             AND DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 2) -- Previous Saturday
),

LabelTotals AS (
    SELECT
        team,
        count_value
    FROM team_counts

    UNION ALL

    SELECT
        'Total' AS team, 
        SUM(count_value) AS count_value 
    FROM team_counts
)

SELECT
    team, 
    count_value 
FROM LabelTotals 

UNION ALL 

SELECT
    'Fully classified emails' AS team, 
    COUNT(DISTINCT essMessageId) AS count_value 
from enhanced_feedback.silver.customer_feedback 
WHERE
    labelData.human_label_count >= 3
   and date(modificationTime) IS NOT NULL
    AND DATE(modificationTime) BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 8) AND DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 2);

### Query 3 : Classification Stats by Priority Level for Current Week

In [0]:
WITH current_week AS (
    SELECT 
        date_sub(current_date(), dayofweek(current_date()) - 1) AS start_date,  -- Most recent Sunday
        date_add(date_sub(current_date(), dayofweek(current_date()) - 1), 6) AS end_date  -- Following Saturday
)
SELECT
    Priority AS Queue,
    SUM(CASE WHEN Agreement = 'All Agreed' THEN sample_count ELSE 0 END) AS `All agreed`,
    SUM(CASE WHEN Agreement = 'Majority' THEN sample_count ELSE 0 END) AS `Majority`,
    SUM(CASE WHEN Agreement = 'Disagreement' THEN sample_count ELSE 0 END) AS `Disagreement`
FROM
    (
        SELECT
          quorum AS Quorum,
          label_agreement AS Agreement,
          labels.priority AS Priority,
          TO_DATE(reports.modificationTime) AS `Report Time`,
          COUNT(*) AS sample_count
        FROM
          (
            SELECT
              ID,
              priority,
              IF(SUM(num_users) >= 3, "Reached Quorum", "Did not reach Quorum") AS quorum,
              CASE
                WHEN COUNT(DISTINCT label) = 1 THEN "All Agreed"
                WHEN MAX(num_users) > SUM(num_users) / 2 THEN "Majority"
                ELSE "Disagreement"
              END AS label_agreement
            FROM
              (
                SELECT
                  ID,
                  label,
                  priority,
                  MIN(`Timestamp`) AS first_label_timestamp,
                  COUNT(DISTINCT Username) AS num_users
                FROM
                  (
                    SELECT
                      human_labels.ID,
                      human_labels.Username,
                      FIRST_VALUE(human_labels.Label) AS label,
                      FIRST_VALUE(human_labels.`Timestamp`) AS `Timestamp`,
                      cf.priority
                    FROM enhanced_feedback.bronze.human_label_parsed hlp
                    LEFT JOIN enhanced_feedback.silver.customer_feedback cf
                      ON hlp.human_labels.ID = cf.sha256_id
                    GROUP BY
                      human_labels.ID,
                      human_labels.Username,
                      cf.priority
                  ) labels
                GROUP BY
                  ID,
                  label,
                  priority
              )
            WHERE
              first_label_timestamp >= (SELECT start_date FROM current_week)
            GROUP BY
              ID,
              priority
          ) labels
        LEFT JOIN enhanced_feedback.silver.customer_feedback reports
          ON labels.ID = reports.sha256_id
        WHERE 
          reports.modificationTime >= (SELECT start_date FROM current_week)
          AND reports.modificationTime < date_add((SELECT end_date FROM current_week), 1)
          AND quorum = 'Reached Quorum'
          AND labels.priority IN ('P0', 'P1','P2')
        GROUP BY
          quorum,
          label_agreement,
          labels.priority,
          TO_DATE(reports.modificationTime)
    ) AS daily_stats
GROUP BY
    Priority
ORDER BY
    Priority;

### Query 4 : Classification Stats by Priority Level for Previous Week

In [0]:
WITH previous_week AS (
    SELECT 
        date_sub(date_sub(CURRENT_DATE, DAYOFWEEK(CURRENT_DATE) - 1), 7) AS start_date,  -- Sunday before current week's Sunday
        date_sub(date_sub(CURRENT_DATE, DAYOFWEEK(CURRENT_DATE) - 1), 1) AS end_date  -- Saturday before current week's Sunday
)
SELECT
    Priority AS Queue,
    SUM(CASE WHEN Agreement = 'All Agreed' THEN sample_count ELSE 0 END) AS `All agreed`,
    SUM(CASE WHEN Agreement = 'Majority' THEN sample_count ELSE 0 END) AS `Majority`,
    SUM(CASE WHEN Agreement = 'Disagreement' THEN sample_count ELSE 0 END) AS `Disagreement`
FROM
    (
        SELECT
          quorum AS Quorum,
          label_agreement AS Agreement,
          labels.priority AS Priority,
          TO_DATE(reports.modificationTime) AS `Report Time`,
          COUNT(*) AS sample_count
        FROM
          (
            SELECT
              ID,
              priority,
              IF(SUM(num_users) >= 3, "Reached Quorum", "Did not reach Quorum") AS quorum,
              CASE
                WHEN COUNT(DISTINCT label) = 1 THEN "All Agreed"
                WHEN MAX(num_users) > SUM(num_users) / 2 THEN "Majority"
                ELSE "Disagreement"
              END AS label_agreement
            FROM
              (
                SELECT
                  ID,
                  label,
                  priority,
                  MIN(`Timestamp`) AS first_label_timestamp,
                  COUNT(DISTINCT Username) AS num_users
                FROM
                  (
                    SELECT
                      human_labels.ID,
                      human_labels.Username,
                      FIRST_VALUE(human_labels.Label) AS label,
                      FIRST_VALUE(human_labels.`Timestamp`) AS `Timestamp`,
                      cf.priority
                    FROM enhanced_feedback.bronze.human_label_parsed hlp
                    LEFT JOIN enhanced_feedback.silver.customer_feedback cf
                      ON hlp.human_labels.ID = cf.sha256_id
                    GROUP BY
                      human_labels.ID,
                      human_labels.Username,
                      cf.priority
                  ) labels
                GROUP BY
                  ID,
                  label,
                  priority
              )
            WHERE
              first_label_timestamp >= (SELECT start_date FROM previous_week)
            GROUP BY
              ID,
              priority
          ) labels
        LEFT JOIN enhanced_feedback.silver.customer_feedback reports
          ON labels.ID = reports.sha256_id
        WHERE 
          reports.modificationTime >= (SELECT start_date FROM previous_week)
          AND reports.modificationTime < date_add((SELECT end_date FROM previous_week), 1)
          AND quorum = 'Reached Quorum'
          AND labels.priority IN ('P0', 'P1','P2')
        GROUP BY
          quorum,
          label_agreement,
          labels.priority,
          TO_DATE(reports.modificationTime)
    ) AS daily_stats
GROUP BY
    Priority
ORDER BY
    Priority;

# Dashboard 1 : labeling stats 
![dashboard_image1.png](./dashboard_image1.png "dashboard_image1.png")

# Dashboard : Analyst labeling Stats
### Query 1: Analyst labeling stats for current week

In [0]:
WITH daily_counts AS (
    SELECT
        human_labels.Username AS analyst,
        DATE(approximateArrivalTimestamp) AS date,
        COUNT(*) AS daily_count
    FROM
        enhanced_feedback.bronze.human_label_parsed AS human_labels
    WHERE
        approximateArrivalTimestamp IS NOT NULL
        AND DATE(approximateArrivalTimestamp) 
        BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1) -- Get Sunday
        AND LEAST(CURRENT_DATE, DATE_ADD(DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 1), 6)) -- Get Saturday
        AND human_labels.Username IS NOT NULL
    GROUP BY
        human_labels.Username, DATE(approximateArrivalTimestamp)
),
weekly_counts AS (
    SELECT
        analyst,
        SUM(daily_count) AS weekly_total
    FROM
        daily_counts
    GROUP BY
        analyst
)
SELECT
    0 as sort_key,
    dc.analyst,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 1 THEN dc.daily_count ELSE 0 END) AS Sunday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 2 THEN dc.daily_count ELSE 0 END) AS Monday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 3 THEN dc.daily_count ELSE 0 END) AS Tuesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 4 THEN dc.daily_count ELSE 0 END) AS Wednesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 5 THEN dc.daily_count ELSE 0 END) AS Thursday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 6 THEN dc.daily_count ELSE 0 END) AS Friday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 7 THEN dc.daily_count ELSE 0 END) AS Saturday,
    SUM(dc.daily_count) AS Total_Analyst_Count_Of_The_Week
FROM
    daily_counts dc
GROUP BY
    sort_key,dc.analyst
UNION ALL
SELECT
    1 as sort_key,
    'Total' AS analyst,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 1 THEN dc.daily_count ELSE 0 END) AS Sunday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 2 THEN dc.daily_count ELSE 0 END) AS Monday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 3 THEN dc.daily_count ELSE 0 END) AS Tuesday,   
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 4 THEN dc.daily_count ELSE 0 END) AS Wednesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 5 THEN dc.daily_count ELSE 0 END) AS Thursday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 6 THEN dc.daily_count ELSE 0 END) AS Friday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 7 THEN dc.daily_count ELSE 0 END) AS Saturday,
    SUM(dc.daily_count) AS Total_Analyst_Count_Of_The_Week
FROM
    daily_counts dc

ORDER BY sort_key, analyst;


### Query 2: Analyst labeling stats for Previous week

In [0]:
WITH daily_counts AS (
    SELECT
        human_labels.Username AS analyst,
        DATE(approximateArrivalTimestamp) AS date,
        COUNT(*) AS daily_count
    FROM
        enhanced_feedback.bronze.human_label_parsed AS human_labels
    WHERE
        approximateArrivalTimestamp IS NOT NULL
        AND DATE(approximateArrivalTimestamp)
        BETWEEN DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 8) -- Get previous Sunday
        AND DATE_SUB(DATE_TRUNC('WEEK', CURRENT_DATE), 2) -- Get previous Saturday
        AND human_labels.Username IS NOT NULL
    GROUP BY
        human_labels.Username, DATE(approximateArrivalTimestamp)
)

SELECT
    0 AS sort_key, 
    dc.analyst,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 1 THEN dc.daily_count ELSE 0 END) AS Sunday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 2 THEN dc.daily_count ELSE 0 END) AS Monday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 3 THEN dc.daily_count ELSE 0 END) AS Tuesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 4 THEN dc.daily_count ELSE 0 END) AS Wednesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 5 THEN dc.daily_count ELSE 0 END) AS Thursday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 6 THEN dc.daily_count ELSE 0 END) AS Friday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 7 THEN dc.daily_count ELSE 0 END) AS Saturday,
    SUM(dc.daily_count) AS Total_Analyst_Count_Of_The_Week
FROM
    daily_counts dc
GROUP BY
    sort_key, dc.analyst 

UNION ALL


SELECT
    1 AS sort_key, 
    'Total' AS analyst,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 1 THEN dc.daily_count ELSE 0 END) AS Sunday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 2 THEN dc.daily_count ELSE 0 END) AS Monday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 3 THEN dc.daily_count ELSE 0 END) AS Tuesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 4 THEN dc.daily_count ELSE 0 END) AS Wednesday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 5 THEN dc.daily_count ELSE 0 END) AS Thursday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 6 THEN dc.daily_count ELSE 0 END) AS Friday,
    SUM(CASE WHEN DAYOFWEEK(dc.date) = 7 THEN dc.daily_count ELSE 0 END) AS Saturday,
    SUM(dc.daily_count) AS Total_Analyst_Count_Of_The_Week
FROM
    daily_counts dc 

ORDER BY sort_key, analyst;

# Dashboard 2: Analyst labeling Stats
![dashboard_image2.png](./dashboard_image2.png "dashboard_image2.png")
![dashboard_image3.png](./dashboard_image3.png "dashboard_image3.png")
![dashboard_image4.png](./dashboard_image4.png "dashboard_image4.png")
![dashboard_image5.png](./dashboard_image5.png "dashboard_image5.png")

# Dashboard : Disagreement Stats Summary
### Query 1 : Disagreement Stats (past 7 days range) used for Disagreement count bar chart

In [0]:
SELECT
  priority AS QueuePriority,
  essMessageId AS ESS_ID,
  modificationTime AS ReportedDate,
  labelData.history[1]['user'] AS first_human_classifier,
  labelData.history[1]['label'] AS first_classification,
  labelData.history[2]['user'] AS second_human_classifier,
  labelData.history[2]['label'] AS second_classification,
  labelData.history[3]['user'] AS third_human_classifier,
  labelData.history[3]['label'] AS third_classification,
  labelData.history[4]['user'] AS disagreement_resolver,
  labelData.history[4]['label'] AS final_classification,
  labelData.human_label_count AS NumberOfHumanLabels,

  -- dis_analyst1
  CASE
    WHEN first_classification != final_classification AND second_classification != final_classification THEN first_human_classifier
    WHEN first_classification != final_classification AND third_classification != final_classification THEN first_human_classifier
    WHEN second_classification != final_classification AND third_classification != final_classification THEN second_human_classifier
    WHEN first_classification != final_classification THEN first_human_classifier
    WHEN second_classification != final_classification THEN second_human_classifier
    WHEN third_classification != final_classification THEN third_human_classifier
    ELSE NULL
  END AS dis_analyst1,
  --  dis_analyst2 
  CASE
    WHEN first_classification != final_classification AND second_classification != final_classification THEN second_human_classifier
    WHEN first_classification != final_classification AND third_classification != final_classification THEN third_human_classifier
    WHEN second_classification != final_classification AND third_classification != final_classification THEN third_human_classifier
    ELSE NULL
  END AS dis_analyst2
FROM
  enhanced_feedback.silver.customer_feedback
WHERE
  labelData.human_label_count >= 3
  AND labelData.history IS NOT NULL
  AND labelData.history[1]['label'] IS NOT NULL
  AND labelData.history[2]['label'] IS NOT NULL
  AND labelData.history[3]['label'] IS NOT NULL
  AND labelData.history[4]['label'] IS NOT NULL 
  AND labelData.history[1]['label'] != labelData.history[2]['label']
  AND labelData.history[1]['label'] != labelData.history[3]['label']
  AND labelData.history[2]['label'] != labelData.history[3]['label']
  AND DATE(modificationTime) >= DATE_SUB(NOW(), 7);

# Query 2 : Disagreements stats per analyst (14 days) used for bar chart and table visualization in the dashboard

In [0]:
WITH DisagreementSamples AS (
  SELECT
    priority AS QueuePriority,
    essMessageId AS ESS_ID,
    modificationTime AS ReportedDate,
    labelData.history AS LabelHistory,
    labelData.history[1]['user'] AS first_human_classifier,
    labelData.history[1]['label'] AS first_classification,
    labelData.history[2]['user'] AS second_human_classifier,
    labelData.history[2]['label'] AS second_classification,
    labelData.history[3]['user'] AS third_human_classifier,
    labelData.history[3]['label'] AS third_classification,
    labelData.history[4]['user'] AS disagreement_resolver,
    labelData.history[4]['label'] AS final_classification,
    labelData.human_label_count AS NumberOfHumanLabels,
    -- dis_analyst1 calculation 
    CASE
      WHEN first_classification != final_classification AND second_classification != final_classification THEN first_human_classifier
      WHEN first_classification != final_classification AND third_classification != final_classification THEN first_human_classifier
      WHEN second_classification != final_classification AND third_classification != final_classification THEN second_human_classifier
      WHEN first_classification != final_classification THEN first_human_classifier
      WHEN second_classification != final_classification THEN second_human_classifier
      WHEN third_classification != final_classification THEN third_human_classifier
      ELSE NULL
    END AS dis_analyst1,
    -- dis_analyst2 calculation 
    CASE
      WHEN first_classification != final_classification AND second_classification != final_classification THEN second_human_classifier
      WHEN first_classification != final_classification AND third_classification != final_classification THEN third_human_classifier
      WHEN second_classification != final_classification AND third_classification != final_classification THEN third_human_classifier
      ELSE NULL
    END AS dis_analyst2
  FROM
    enhanced_feedback.silver.customer_feedback
  WHERE
    labelData.human_label_count >= 3
    AND labelData.history IS NOT NULL
    AND labelData.history[1]['label'] IS NOT NULL
    AND labelData.history[2]['label'] IS NOT NULL
    AND labelData.history[3]['label'] IS NOT NULL
    AND labelData.history[4]['label'] IS NOT NULL
    AND labelData.history[1]['label'] != labelData.history[2]['label']
    AND labelData.history[1]['label'] != labelData.history[3]['label']
    AND labelData.history[2]['label'] != labelData.history[3]['label']
    AND DATE(modificationTime) >= DATE_SUB(NOW(), 14)
)

SELECT
  QueuePriority,
  ESS_ID,
  ReportedDate,
  LabelHistory,
  first_human_classifier,
  first_classification,
  second_human_classifier,
  second_classification,
  third_human_classifier,
  third_classification,
  disagreement_resolver,
  final_classification,
  NumberOfHumanLabels,

  dis_analyst1 AS InvolvedAnalyst, 
  CASE
    WHEN dis_analyst1 = first_human_classifier THEN first_classification
    WHEN dis_analyst1 = second_human_classifier THEN second_classification
    WHEN dis_analyst1 = third_human_classifier THEN third_classification
    ELSE NULL 
  END AS dis_classification 
FROM
  DisagreementSamples
WHERE
  dis_analyst1 IS NOT NULL 

UNION ALL
SELECT
  QueuePriority,
  ESS_ID,
  ReportedDate,
  LabelHistory,
  first_human_classifier,
  first_classification,
  second_human_classifier,
  second_classification,
  third_human_classifier,
  third_classification,
  disagreement_resolver,
  final_classification,
  NumberOfHumanLabels,
  dis_analyst2 AS InvolvedAnalyst, 
  CASE
    WHEN dis_analyst2 = first_human_classifier THEN first_classification
    WHEN dis_analyst2 = second_human_classifier THEN second_classification
    WHEN dis_analyst2 = third_human_classifier THEN third_classification
    ELSE NULL 
  END AS dis_classification 
FROM
  DisagreementSamples
WHERE
  dis_analyst2 IS NOT NULL 
  AND dis_analyst1 != dis_analyst2; 


# Dashboard 3 : Disagreement Stats Summary
![dashboard_image6.png](./dashboard_image6.png "dashboard_image6.png")
![dashboard_image7.png](./dashboard_image7.png "dashboard_image7.png")

# Dashboard : SLA stats summary
### Query 1 : SLA nearing samples (For last the 40 to 48 hrs) used for the table visualization

In [0]:
WITH sha256_id_lookup AS (
   SELECT DISTINCT sha256_id from enhanced_feedback.silver.labeling_ui
),
P0_samples AS (
    SELECT DISTINCT
        cf.essMessageId,
        cf.labelData.report_date,
        cf.labelData.human_label_count
    FROM 
        enhanced_feedback.silver.customer_feedback cf
        LEFT JOIN sha256_id_lookup sli 
        ON cf.sha256_id = sli.sha256_id
    WHERE 
        cf.priority = 'P0'  
        AND cf.labelData.human_label_count < 3  
        AND cf.labelData.report_date >= current_timestamp() - INTERVAL '48 hours'
        AND cf.labelData.report_date <= current_timestamp() - INTERVAL '40 hours'
)
SELECT essMessageId, report_date, human_label_count
FROM P0_samples;

### Query 2 : Current week sla expiry

In [0]:
-- counts for 'Analyzed but missed SLA' sends
CREATE OR REPLACE TEMPORARY VIEW p0_missed_sla_sends
AS
SELECT
  DATE(ed.report_timestamp) AS received_date,
  COUNT(DISTINCT ed.sha256_id) AS missed_sla_sends_count
FROM enhanced_feedback.gold.email_digest ed
LEFT JOIN enhanced_feedback.silver.labeling_ui ui ON ed.sha256_id = ui.sha256_id
WHERE ui.priority = 'P0'
  AND ed.report_timestamp > DATE_SUB(NOW(), 30) 
  AND (
    sent_at_timestamp IS NOT NULL AND analysis_timestamp IS NOT NULL AND date_sub(analysis_timestamp, 2) > report_timestamp
  )
GROUP BY DATE(ed.report_timestamp);

--   CURRENT week (Sunday to Saturday)
WITH DateSeries AS (
  SELECT date_add(date_sub(current_date(), dayofweek(current_date()) - 1), n) AS report_date
  FROM (SELECT explode(sequence(0, 6)) AS n) 
)

SELECT
  date_format(report_date, 'EEEE') AS day_name,
  COALESCE(ps.missed_sla_sends_count, 0) AS sla_expired_count,
  ds.report_date AS reported_date
FROM
  DateSeries ds
LEFT JOIN
  p0_missed_sla_sends ps ON ds.report_date = ps.received_date
ORDER BY
  dayofweek(ds.report_date); 

### Query 3 : Previous week sla expiry

In [0]:
-- counts for 'Analyzed but missed SLA' sends
CREATE OR REPLACE TEMPORARY VIEW p0_missed_sla_sends
AS
SELECT
  DATE(ed.report_timestamp) AS received_date,
  COUNT(DISTINCT ed.sha256_id) AS missed_sla_sends_count
FROM enhanced_feedback.gold.email_digest ed
LEFT JOIN enhanced_feedback.silver.labeling_ui ui ON ed.sha256_id = ui.sha256_id
WHERE ui.priority = 'P0'
  AND ed.report_timestamp > DATE_SUB(NOW(), 30) 
  AND (
    sent_at_timestamp IS NOT NULL AND analysis_timestamp IS NOT NULL AND date_sub(analysis_timestamp, 2) > report_timestamp 
  )
GROUP BY DATE(ed.report_timestamp);

--  PREVIOUS week (Sunday to Saturday)
WITH DateSeries AS ( 
  SELECT date_add(date_sub(DATE_TRUNC('WEEK', CURRENT_DATE), 8), n) AS report_date 
  FROM (SELECT explode(sequence(0, 6)) AS n) 
)
SELECT
  date_format(report_date, 'EEEE') AS day_name,
  COALESCE(ps.missed_sla_sends_count, 0) AS sla_expired_count,
  ds.report_date AS reported_date
FROM
  DateSeries ds 
LEFT JOIN
  p0_missed_sla_sends ps ON ds.report_date = ps.received_date 
ORDER BY
  dayofweek(ds.report_date);

# Query 4 : sla stats (current week)

In [0]:
-- samples that missed SLA in the current week
SELECT
  cf.essMessageId,
  DATE(cf.labelData.report_date) AS report_date, 
  cf.labelData.history[1]['label'] AS first_classification,
  cf.labelData.history[2]['label'] AS second_classification,
  cf.labelData.history[3]['label'] AS third_classification,
  cf.priority
  --cf.labelData.history AS labelHistory 
FROM
  enhanced_feedback.silver.customer_feedback cf
JOIN
  enhanced_feedback.gold.email_digest ed ON cf.sha256_id = ed.sha256_id 
WHERE
  -- Filter for the 'Analyzed but missed SLA' condition based on email_digest data
  ed.sent_at_timestamp IS NOT NULL
  AND ed.analysis_timestamp IS NOT NULL
  AND date_sub(ed.analysis_timestamp, 2) > ed.report_timestamp 

  -- Filter for the date range of the CURRENT week based on report_date
  AND DATE(cf.labelData.report_date) BETWEEN date_sub(current_date(), dayofweek(current_date()) - 1) AND date_add(current_date(), 7 - dayofweek(current_date()))
ORDER BY cf.labelData.report_date; 

# Query 5 : sla stats (previous week)

In [0]:
-- samples that missed SLA in the previous week
SELECT
  cf.essMessageId,
  DATE(cf.labelData.report_date) AS report_date, 
  cf.labelData.history[1]['label'] AS first_classification,
  cf.labelData.history[2]['label'] AS second_classification,
  cf.labelData.history[3]['label'] AS third_classification,
  cf.priority
  --cf.labelData.history AS labelHistory 
FROM
  enhanced_feedback.silver.customer_feedback cf
JOIN
  enhanced_feedback.gold.email_digest ed ON cf.sha256_id = ed.sha256_id 
WHERE
  -- Filter for the 'Analyzed but missed SLA' condition based on email_digest data
  ed.sent_at_timestamp IS NOT NULL
  AND ed.analysis_timestamp IS NOT NULL
  AND date_sub(ed.analysis_timestamp, 2) > ed.report_timestamp 

  AND DATE(cf.labelData.report_date) BETWEEN date_sub(next_day(current_date(), 'SU'), 14) AND date_sub(next_day(current_date(), 'SU'), 8)
ORDER BY cf.labelData.report_date; 

# Dashboard 4 : sla stats summary
![dashboard_image8.png](./dashboard_image8.png "dashboard_image8.png")
![dashboard_image9.png](./dashboard_image9.png "dashboard_image9.png")